# Import the Necessary Libraries/Packages

In [1]:
# Import the necessary libraries/packages
import pandas as pd
import numpy as np

# Instantiate the Necessary Constants

In [2]:
# Instantiate the necessary constants
#TARGET_USERS = [6]
TARGET_USERS = [196, 186, 22, 244, 166, 298, 115, 253, 305, 6]

# Load/Clean the Necessary Data Sets

In [3]:
# Read in the .csv file with the MLP predictions
MLP_preds = pd.read_csv("data/mlp_results.csv")
MLP_preds.head()

,user_id,item_id,predicted_rating
0,196,242,4.0
1,186,302,5.0
2,22,377,1.0
3,244,51,4.0
4,166,346,4.0


In [4]:
# Read in the .csv file with the CF predictions
CF_preds = pd.read_csv("data/10_CF_results.csv")
CF_preds.head()

,UserID,MovieID,PredictedScore
0,196,Rating.302,3.707343
1,196,Rating.377,3.707343
2,196,Rating.51,3.707343
3,196,Rating.346,3.707343
4,196,Rating.474,3.707343


In [5]:
# Modify the feature "MovieID" of CF_preds to remove "Rating." from each observation
CF_preds["MovieID"] = CF_preds["MovieID"].str[7:].astype(int)
CF_preds.head()

,UserID,MovieID,PredictedScore
0,196,302,3.707343
1,196,377,3.707343
2,196,51,3.707343
3,196,346,3.707343
4,196,474,3.707343


# Implement the Hybrid MLP-CF Model

## Pull the CF Ratings for Each User

In [6]:
# Separate CF_preds into DataFrames for each user
CF_user_filt_dfs = []

# Iterate over the list of target users
for user in TARGET_USERS:

    # Filter CF_preds by the user
    CF_preds_filt = CF_preds[CF_preds["UserID"]==user]

    # Sort the filtered data by movie ID from lowest to highest
    CF_preds_filt_sort = CF_preds_filt.sort_values(by=["MovieID"], ascending=True)

    # Append the filtered, sorted data to the list of DataFrames
    CF_user_filt_dfs.append(CF_preds_filt_sort)

## Pull the MLP Ratings for the Unrated Movies for Each User

In [7]:
# Filter MLP_preds based on each target user's missing ratings
MLP_user_filt_dfs = []

# Iterate over each CF user DataFrame
for user_df in CF_user_filt_dfs:
    # Gauge the movies in the MLP predictions data that are NOT in each CF user DataFrame
    CF_movies = user_df["MovieID"]
    MLP_movies = MLP_preds["item_id"]

    res = list(set(MLP_movies) - set(CF_movies))

    # Filter out the necessary movies from MLP_pred
    MLP_preds_filt = MLP_preds.loc[~MLP_preds["item_id"].isin(res)]

    # Sort the filtered data by movie ID from lowest to highest
    MLP_preds_filt_sort = MLP_preds_filt.sort_values(by=["item_id"], ascending=True)

    # Append the filtered, sorted data to the list of DataFrames
    MLP_user_filt_dfs.append(MLP_preds_filt_sort)

## Obtain the Movie-Highest Rating Pair for Each User

In [8]:
# Populate a dictionary with the highest rating and its associated movie for each user
CF_MLP_merged_dfs = []
target_users_recs = {}

for i in range(len(CF_user_filt_dfs)):
    for j in range(len(MLP_user_filt_dfs)):
        if i == j:
            # Merge the CF and MLP DataFrames for each user on movie ID
            CF_MLP_merged = pd.merge(CF_user_filt_dfs[i], MLP_user_filt_dfs[j], left_on="MovieID", right_on="item_id")

            # Filter for rows where CF predicted rating > MLP predicted rating
            CF_MLP_merged_filt = CF_MLP_merged.loc[CF_MLP_merged["PredictedScore"] > CF_MLP_merged["predicted_rating"]]

            # Sort the filtered DataFrame by CF predicted rating from highest to lowest
            CF_MLP_merged_filt_sort = CF_MLP_merged_filt.sort_values(by=["PredictedScore"], ascending=False)
            CF_MLP_merged_dfs.append(CF_MLP_merged_filt_sort)

            # Update the dictionary with the highest CF predicted rating and its movie
            target_users_recs[TARGET_USERS[i]] = {"Movie ID": CF_MLP_merged_filt_sort["MovieID"].iloc[0], "Highest Rating": CF_MLP_merged_filt_sort["PredictedScore"].iloc[0]}

# Display the users' results
print(target_users_recs)

{196: {'Movie ID': 357, 'Highest Rating': 4.24656393477016}, 186: {'Movie ID': 286, 'Highest Rating': 4.60968426781676}, 22: {'Movie ID': 183, 'Highest Rating': 4.33432822186426}, 244: {'Movie ID': 483, 'Highest Rating': 4.67168002214678}, 166: {'Movie ID': 228, 'Highest Rating': 3.89983271600903}, 298: {'Movie ID': 192, 'Highest Rating': 4.45161172732633}, 115: {'Movie ID': 169, 'Highest Rating': 5.0}, 253: {'Movie ID': 195, 'Highest Rating': 4.127183474125}, 305: {'Movie ID': 23, 'Highest Rating': 4.35571545558406}, 6: {'Movie ID': 654, 'Highest Rating': 4.67180577723681}}


User 196:
-few movies this user rated as a 5 include 655 (Stand by Me (1986); Adventure/Comedy/Drama), 67 (Ace Ventura: Pet Detective (1994); Comedy), and 663 (Being There (1979); Comedy) --> this user seems to like comedies 
-the unrated movie with the highest rating is movie 357 (e.g. One Flew Over the Cuckoo's Nest (1975); Drama) with a rating of roughly 4.25
-One Flew Over the Cuckoo's Nest serves as a proper movie recommendation for User 196

User 186
-few movies this user rated as a 5 include 566 (Clear and Present Danger (1994); Action/Adventure/Thriller), 470 (Tombstone (1993); Western), and 925 (Unforgettable (1996); Sci-Fi/Thriller) --> this user seems to like movies that keep them on the edge of their seat
-the unrated movie with the highest rating is movie 286 (e.g. The English Patient (1996); Drama/Romance/War) with a rating of roughly 4.61
-The English Patient serves as a proper movie recommendation for User 186

User 22
-few movies this user rated as a 5 include 128 (Supercop (1992); Action/Thriller), 258 (Contact (1997); Drama/Sci-Fi), and 510 (The Magnificent Seven (1954); Action/Drama/Western) --> this user seems to like action movies
-the unrated movie with the highest rating is movie 183 (e.g. Alien (1979); Action/Horror/Sci-Fi/Thriller) with a rating of roughly 4.33
-Alien serves as a proper movie recommendation for User 22

User 244
-few movies this user rated as a 5 include 154 (Monty Python's Life of Brian (1979); Comedy), 89 (Blade Runner (1982); Film-Noir/Sci-Fi), and 652 (Rosencrantz and Guildenstern Are Dead (1990); Comedy/Drama) --> this user seems to like comedies
-the unrated movie with the highest rating is movie 483 (e.g. Casablanca (1942); Drama/Romance/War) with a rating of roughly 4.67
-Casablanca serves as a proper movie recommendation for User 244

User 166
-few movies this user rated as a 5 include 328 (Conspiracy Theory (1997); Action/Mystery/Romance/Thriller), 322 (Murder at 1600 (1997); Mystery/Thriller), and 300 (Air Force One (1997); Action/Thriller) --> this user seems to like thrillers  
-the unrated movie with the highest rating is movie 228 (e.g. Star Trek: The Wrath of Khan (1982); Action/Adventure/Sci-Fi) with a rating of roughly 3.90
-Star Trek: The Wrath of Khan serves as a proper movie recommendation for User 166

User 298
-few movies this user rated as a 5 include 8 (Babe (1995); Children's/Comedy/Drama), 498 (The African Queen (1951); Action/Adventure/Romance/War), and 496 (It's a Wonderful Life (1946); Drama) --> this user seems to like feel-good movies
-the unrated movie with the highest rating is movie 192 (e.g. Raging Bull (1980); Drama) with a rating of roughly 4.45
-Raging Bull serves as a proper movie recommendation for User 298

User 115
-few movies this user rated as a 5 include 8 (Babe (1995); Children's/Comedy/Drama), 127 (The Godfather (1972); Action/Crime/Drama), and 234 (Jaws (1975); Action/Horror) --> this user seems to like movies that keep them on the edge of their seat
-the unrated movie with the highest rating is movie 169 (e.g. The Wrong Trousers (1993); Animation/Comedy) with a rating of 5
-The Wrong Trousers serves as a proper movie recommendation for User 115

User 253
-few movies this user rated as a 5 include 465 (The Jungle Book (1994); Adventure/Children's/Romance), 510 (The Magnificent Seven (1954); Action/Drama/Western), and 183 (e.g. Alien (1979); Action/Horror/Sci-Fi/Thriller) --> this user seems to like movies that keep their attention
-the unrated movie with the highest rating is movie 195 (e.g. The Terminator (1984); Action/Sci-Fi/Thriller) with a rating of roughly 4.13
-The Terminator serves as a proper movie recommendation for User 253

User 305
-few movies this user rated as a 5 include 427 (To Kill a Mockingbird (1962); Drama), 483 (e.g. Casablanca (1942); Drama/Romance/War), and 50 (Star Wars (1977); Action/Adventure/Romance/Sci-Fi/War) --> this user seems to like dramas
-the unrated movie with the highest rating is movie 23 (e.g. Taxi Driver (1976); Drama/Thriller) with a rating of roughly 4.36
-Taxi Driver serves as a proper movie recommendation for User 305

User 6
-few movies this user rated as a 5 include 14 (Il Postino (1994); Drama/Romance), 98 (The Silence of the Lambs (1991); Drama/Thriller), and 492 (East of Eden (1955); Drama) --> this person seems to like dramas
-the unrated movie with the highest rating is movie 654 (e.g. Chinatown (1974); Film-Noir/Mystery/Thriller) with a rating of roughly 4.67
-Chinatown serves as a proper movie recommendation for User 6